<a href="https://colab.research.google.com/github/WeitingGG/Music-Generation/blob/master/music_project_with_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from numpy.random import seed
seed(5)
import tensorflow as tf
tf.random.set_seed(5)
#dealing with midi   files
from music21 import * 


#array   processing
import numpy as np     
import os

#random   number generator
import random         

#keras   for building deep learning model
from keras.layers import * 
from keras.models import *
import keras.backend as K
def read_midi(file):
  notes=[]
  notes_to_parse = None

 #parsing a midi   file
  midi = converter.parse(file)
 #grouping based on   different instruments
  s2 = instrument.partitionByInstrument(midi)

 #Looping over all   the instruments
  for part in s2.parts:
 #select elements   of only piano
    if 'Piano' in str(part): 
      notes_to_parse = part.recurse() 
 #finding   whether a particular element is note or a chord
      for element in notes_to_parse:
        if isinstance(element, note.Note):
          notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))

  return notes

In [0]:
path="/content/drive/My Drive/music_sample"
#read   all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]
print(files)

['schumm-1.mid', 'schubert_D935_2.mid']


In [0]:
#reading   each midi file
all_notes=[]
for i in (0,1):
  filepath="/content/drive/My Drive/music_sample/"+files[i]
  print(filepath)
  all_notes.append(read_midi(filepath))
  print(all_notes)

/content/drive/My Drive/music_sample/schumm-1.mid
[['G4', 'G3', 'A4', 'A3', 'G4', 'G3', 'E4', 'E3', 'C4', 'C3', 'G4', 'G3', 'E5', 'E4', 'E5', 'E4', 'C5', 'C4', 'G4', 'F4', 'E4', 'C4', 'G3', 'C4', 'E4', 'G4', '4.7', 'G2', '2.5.7', 'A2', 'B2', '0.4.7', 'C3', '7.11.2', 'G2', '0.4.7', 'C3', '3.7', 'G2', '2.5.7', 'A2', 'B2', '0.3.7', 'C3', '7.11.2', 'G2', '0.3.7', 'C3', '10.2.5', 'B-2', '10.2.5', 'B-2', '3.7.10', 'E-3', '3.7.10', 'E-3', '2.6.9', 'D3', '2.6.9', 'D3', '7.10.2', 'G3', '7.10.2', 'G3', '7.11.2', '5.7', '7.11.2', '5.7', '5.11', '2.7', '0.4', '7.0', 'G4', 'G3', 'A4', 'A3', 'G4', 'G3', 'E4', 'E3', 'C4', 'C3', 'G4', 'G3', 'E5', 'E4', 'E5', 'E4', 'C5', 'C4', 'G4', 'F4', 'E4', 'C4', 'G3', 'C4', 'E4', 'G4', '4.7', 'G2', '2.5.7', 'A2', 'B2', '0.4.7', 'C3', '7.11.2', 'G2', '0.4.7', 'C3', '3.7', 'G2', '2.5.7', 'A2', 'B2', '0.3.7', 'C3', '7.11.2', 'G2', '0.3.7', 'C3', '10.2.5', 'B-2', '10.2.5', 'B-2', '3.7.10', 'E-3', '3.7.10', 'E-3', '2.6.9', 'D3', '2.6.9', 'D3', '7.10.2', 'G3', '7.10.2',

In [0]:
#notes   and chords of all the midi files
notes = [element for notes in all_notes for element in notes]

In [0]:
#length   of a input sequence
no_of_timesteps = 128      

n_vocab = len(set(notes))  
pitch = sorted(set(item for item in notes))  

#no. of   unique notes
note_to_int = dict((note, number) for number, note in enumerate(pitch))  
#all   the unique notes
X = []
y = []
for notes in all_notes:
  for i in range(0, len(notes) - no_of_timesteps, 1):
    input_ = notes[i:i + no_of_timesteps]
    output = notes[i + no_of_timesteps]
    X.append([note_to_int[note] for note in input_])
    y.append(note_to_int[output])
X = np.reshape(X, (len(X), no_of_timesteps, 1))
#normalizing   the inputs

In [0]:
X = X / float(n_vocab)
print(X)
def lstm():
  model = Sequential()
  model.add(LSTM(128,return_sequences=True))
  model.add(LSTM(128))
  model.add(Dense(256))
  model.add(Activation('relu'))
  model.add(Dense(n_vocab))
  model.add(Activation('softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model
K.clear_session()

def simple_wavenet():
  no_of_kernels=64
  num_of_blocks= int(np.sqrt(no_of_timesteps)) - 1 

  model = Sequential()
  for i in range(num_of_blocks):
    model.add(Conv1D(no_of_kernels,3,dilation_rate=(2**i),padding='causal',activation='relu'))
  model.add(Conv1D(1, 1, activation='relu', padding='causal'))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dense(n_vocab, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
  return model

[[[0.98571429]
  [0.97857143]
  [0.56428571]
  ...
  [0.77857143]
  [0.25      ]
  [0.77857143]]

 [[0.97857143]
  [0.56428571]
  [0.55714286]
  ...
  [0.25      ]
  [0.77857143]
  [0.22142857]]

 [[0.56428571]
  [0.55714286]
  [0.98571429]
  ...
  [0.77857143]
  [0.22142857]
  [0.75      ]]

 ...

 [[0.64285714]
  [0.75714286]
  [0.60714286]
  ...
  [0.47857143]
  [0.02142857]
  [0.41428571]]

 [[0.75714286]
  [0.60714286]
  [0.47857143]
  ...
  [0.02142857]
  [0.41428571]
  [0.06428571]]

 [[0.60714286]
  [0.47857143]
  [0.78571429]
  ...
  [0.41428571]
  [0.06428571]
  [0.47142857]]]


In [0]:

from keras.callbacks import ModelCheckpoint
def train(model, network_input, network_output):
    history=[]
    filepath = "{loss:.4f}.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]
    train_history=model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)
    train_history
    loss=train_history.history['loss']
    print(loss)
    return(loss)


model=simple_wavenet()
netinput=X
y = np.array(y)
netoutput=y
Loss=train(model,netinput,netoutput)


Epoch 1/5
3454/3454 [==============================] - 2s 687us/step - loss: 4.9272
Epoch 2/5
3454/3454 [==============================] - 2s 535us/step - loss: 4.8668
Epoch 3/5
3454/3454 [==============================] - 2s 535us/step - loss: 4.7523
Epoch 4/5
3454/3454 [==============================] - 2s 536us/step - loss: 4.6217
Epoch 5/5
3454/3454 [==============================] - 2s 534us/step - loss: 4.5321
[4.927194822849258, 4.866803285767732, 4.752287203408281, 4.62171313330557, 4.532084293288168]


In [0]:
best=round(min(Loss),4)

4.5321


In [0]:
def generate_music(model, pitch, no_of_timesteps, pattern):

    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    prediction_output = []

 # generate 50   elements
    for note_index in range(50):
 #reshaping   array to feed into model
        input_ = np.reshape(pattern, (1, len(pattern), 1))
 #predict   the probability and choose the maximum value
        proba = model.predict(input_, verbose=0)
        index = np.argmax(proba)

        pred = int_to_note[index]
        prediction_output.append(pred)
        pattern = list(pattern)
        pattern.append(index/float(n_vocab))
 #leave   the first value at index 0
        pattern = pattern[1:len(pattern)]
        print("generate succeed")

    return prediction_output

In [0]:

def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

 # create note and   chord objects based on the values generated by the model
    for pattern in prediction_output:
 #   pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
 #   pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

 #   Specify duration between 2 notes
        offset+= 0.5
       # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='NewMusic.mid')
    print("convert succeed")

In [0]:

#Select   random chunk for the first iteration
start = np.random.randint(0, len(X)-1)
pattern = X[start]


In [0]:
#load   the best model
bestfile='/content/'+str(best)+'.hdf5'
model=load_model(bestfile)

In [0]:
#generate   and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
print("done")
convert_to_midi(music)
print("done1")

generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
generate succeed
done
convert succeed
done1
